## Part 1: Preprocessing

In [48]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [49]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [50]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

In [51]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'DistanceFromHome', 'Education', 'JobLevel', 'WorkLifeBalance', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'YearsAtCompany']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
print(X_df.dtypes)

Age                   int64
DistanceFromHome      int64
Education             int64
JobLevel              int64
WorkLifeBalance       int64
NumCompaniesWorked    int64
PercentSalaryHike     int64
StockOptionLevel      int64
TotalWorkingYears     int64
YearsAtCompany        int64
dtype: object


In [52]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [53]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


In [54]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)

In [55]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder()

# Fit the encoder to the training data
y_train_dept = dept_encoder.fit_transform(y_train[['Department']]).toarray()

# Create two new variables by applying the encoder
# to the training and testing data
y_test_dept = dept_encoder.transform(y_test[['Department']]).toarray()

KeyError: "None of [Index(['department'], dtype='object')] are in the [columns]"

In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder()

# Fit the encoder to the training data
y_train_attrition = attrition_encoder.fit_transform(y_train[['attrition']]).toarray()

# Create two new variables by applying the encoder
# to the training and testing data
y_test_attrition = attrition_encoder.transform(y_test[['attrition']]).toarray()


## Part 2: Create, Compile, and Train the Model

In [10]:

# Find the number of columns in the X training data.

num_features = X_train_scaled.shape[1]
# Create the input layer
from tensorflow.keras.layers import Input
input_layer = Input(shape=(num_features,))

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_branch = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
dept_output = Dense(y_train_dept.shape[1], activation='softmax')(dept_branch)

In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = Dense(y_train_attrition.shape[1], activation='softmax')(attrition_branch)

In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summarize the model
model.summary()

In [14]:
# Train the model
history = model.fit(X_train_scaled, [y_train_dept, y_train_attrition], epochs=100, batch_size=32, validation_split=0.2)

In [15]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [y_test_dept, y_test_attrition])


In [16]:
# Print the accuracy for both Department and attrition
print(f"Department Accuracy: {results[3]}")
print(f"Attrition Accuracy: {results[4]}")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric if the data is imbalanced 
2. Softmax is sitible for multiclass needs. 
3. Use a more complex feature selction and expierement with different architicltures 